---
jupyter:
  jupytext:
    text_representation:
      extension: .md
      format_name: markdown
      format_version: '1.3'
      jupytext_version: 1.16.0
  kernelspec:
    display_name: Python 3
    language: python
    name: python3
---

<!-- #region id="6e93f194" -->
# Table des matières
- [Exemple de régression par forêt aléatoire](#exemple-de-régression-par-forêt-aléatoire)
      - [Ce que nous allons voir dans cette section](#ce-que-nous-allons-voir-dans-cette-section)
  - [Lecture, affichage et préparation des données](#lecture-affichage-et-préparation-des-données)
      - [Lecture des données originales de l'étude.](#lecture-des-données-originales-de-létude)
      - [Génération d'un DataFrame et identification des noms de variables.](#génération-dun-dataframe-et-identification-des-noms-de-variables)
      - [Affichage de la distribution des prix des maisons.](#affichage-de-la-distribution-des-prix-des-maisons)
      - [Affichage des cinq premières lignes des variables](#affichage-des-cinq-premières-lignes-des-variables)
      - [Génération des ensembles d'entraînement ($80~\%$) et de test ($20~\%$)](#génération-des-ensembles-dentraînement-80%25-et-de-test-20%25)
      - [Normalisation des données](#normalisation-des-données)
  - [Entraînement d'un régresseur par forêt aléatoire](#entraînement-dun-régresseur-par-forêt-aléatoire)
      - [Sélection du type de régresseur.](#sélection-du-type-de-régresseur)
      - [Entraînement du régresseur avec les données d'entraînement](#entraînement-du-régresseur-avec-les-données-dentraînement)
      - [Prédiction du prix des maisons pour les deux ensembles de données](#prédiction-du-prix-des-maisons-pour-les-deux-ensembles-de-données)
  - [Affichage des résultats](#affichage-des-résultats)
      - [Affichage de la valeur du R2](#affichage-de-la-valeur-du-r2)
      - [Affichage des valeurs prédites et réelles des maisons pour l'ensemble de test.](#affichage-des-valeurs-prédites-et-réelles-des-maisons-pour-lensemble-de-test)
  - [Identification des variables les plus importantes déterminant le prix d'une maison](#identification-des-variables-les-plus-importantes-déterminant-le-prix-dune-maison)
    - [Affichage du diagrame d'importance des variables du jeu de données](#affichage-du-diagrame-dimportance-des-variables-de-la-base-de-données)
    - [Comment varie le prix des maisons en fonction des deux variables les plus importantes?](#comment-varie-le-prix-des-maisons-en-fonction-des-deux-variables-les-plus-importantes)
      - [Sélection des variables d'intérêt](#sélection-des-variables-dintérêt)
      - [Affichage de l'effet de chaque variable sur le prix des maisons.](#affichage-de-leffet-de-chaque-variable-sur-le-prix-des-maisons)
  - [Exercice](#exercice)
  - [Exercice](#exercice-1)
<!-- #endregion -->

<!-- #region id="20d14f24" -->
---
# Exemple de régression par forêt aléatoire
---
<!-- #endregion -->

<!-- #region id="0a9aa7ed" -->
<p>&nbsp;</p>
<div align="center">
    <img src= "../images/boston-houses.jpeg"  width="500" />
    <div>
    <font size="0.5">Image Source: https://www.flickr.com/photos/romanboed/49365609358/</font>
    </div>
</div>
<!-- #endregion -->

<!-- #region id="f2ea64bc" -->
Dans cette section, nous allons appliquer la régression par forêt aléatoire afin de prédire le prix, dans
les années 70, des maisons dans la région de Boston au Massachusetts (États-Unis).

C'est un excellent exemple souvent utilisé, en apprentissage automatique, pour se familiariser avec la difficulté
de faire de la prédiction en se basant sur des données hétéroclites variant de façon discontinue.

Comme on l'a vu dans les précédentes sections, la régression sert à prédire une *réponse* $y$, qui est
une variable **numérique continue**, en fonction de plusieurs facteurs/variables $x_{i}$

$$y=f(x_{1}, \cdots, x_{N}, \Theta)$$

où $\Theta$ représente l'ensemble des paramètres de la fonction $f$. Dans plusieurs domaines, comme en physique, on peut
dériver analytiquement une telle formule basée sur les lois de la nature. Dans les sciences de la vie et les sciences
sociales, c'est généralement impossible dû au grand nombre de phénomènes impliqués lors de la prise de données. La
modélisation par forêt aléatoire permet de se libérer de cette contrainte irréaliste en modélisation analytique. De
par sa nature, la forêt aléatoire permet d'explorer naturellement un très grand nombre d'interactions entre les
différentes variables sans avoir à les spécifier explicitement dans un modèle mathématique. C'est une des
multiples raisons pour lesquelles elles performent souvent mieux que d'autres méthodes non-linéaires.

#### Ce que nous allons voir dans cette section

Nous allons initialement modéliser le prix des maisons en fonction des variables du jeu de données utilisée. Le
but étant de pouvoir faire des prédictions de prix pour d'autres maisons de Boston, à la même époque.

Nous allons ensuite déterminer lesquelles, parmi ces variables, sont les plus utiles pour faire des
prédictions. Nous allons enfin examiner leur effet sur le prix des maisons.
<!-- #endregion -->



In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
import seaborn as sns

sns.set(color_codes=True)

%matplotlib inline

# Pour la reproductibilité des résultats
np.random.seed(seed)



<!-- #region id="72520c76" -->
## Lecture, affichage et préparation des données

Le  jeu de données contient 13 variables/facteurs. Les données sont toutes numériques. Elles ont été collectées
dans les années 70 par le recensement des États-Unis dans la région de Boston au Massachusetts.

Voici la liste des facteurs mesurés. Le but est de déterminer comment varie le prix des maisons (dans les années 70!)
en fonction de ces facteurs:


- CRIM:  taux de criminalité, 
- ZN:  proportion de terrain résidentiel zoné pour les lots de plus de 25 000 pi2, 
- INDUS:  proportion d'acres commerciales non commerciales par ville, 
- CHAS:  indicateur de proximité avec la Charles River (= 1 si près la rivière ; 0 sinon), 
- NOX:  concentration d'oxydes nitriques (parties par 10 millions), 
- RM:  nombre moyen de pièces par logement, 
- AGE:  proportion d'unités construites avant 1940 occupées par leur propriétaire, 
- DIS:  distances pondérées à cinq centres d'emploi de Boston, 
- RAD:  indice d'accessibilité aux autoroutes radiales, 
- TAX:  taux d'impôt foncier sur la valeur totale par tranche de ~\$10 000, 
- PTRATIO:  ratio élèves-enseignant par ville, 
- B:  $1000(Bk - 0.63)^2$ où Bk est la proportion de personnes d'ascendance afro-américaine, 
- LSTAT:  pourcentage des classes défavorisées dans la population, 
- MEDV:  valeur médiane des logements occupés par leur propriétaire en k~\$. 



#### Lecture des données originales de l'étude
<!-- #endregion -->



In [ ]:
data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
y = raw_df.values[1::2, 2]



<!-- #region id="8205cb8e" -->
#### Génération d'un DataFrame et identification des noms de variables
<!-- #endregion -->



In [ ]:
X = pd.DataFrame(
    data,
    columns=[
        'CRIM',
        'ZN',
        'INDUS',
        'CHAS',
        'NOX',
        'RM',
        'AGE',
        'DIS',
        'RAD',
        'TAX',
        'PTRATIO',
        'B',
        'LSTAT',
    ],
)

# Liste des facteurs utilisés
feature_list = list(X.columns)



<!-- #region id="f3c9e153" -->
#### Affichage de la distribution des prix des maisons
<!-- #endregion -->



In [ ]:
plt.figure(figsize=(5, 4))
plt.hist(y, 20)
plt.title('Histogramme du prix des maisons')
plt.xlabel('price (k$)')
plt.ylabel('N')
plt.show()



<!-- #region id="df7a36bc" -->
C'est la quantité que l'on désire prédire en fonction des variables $x_{i}$ précédents. Le prix maximum des
maisons est de 50k$! C'était dans les années 70..

#### Affichage des cinq premières lignes des variables
<!-- #endregion -->



In [ ]:
X.head()



<!-- #region id="618d5126" -->
On voit que les valeurs couvrent différents ordres de grandeur. Il faudra les normaliser pour cette raison. Il faut toutefois mentionner que les arbres décisionnels et les forêts aléatoires ne sont pas affectés par la normalisation. Néanmoins, c'est une bonne habitude de normaliser ses données; autant commencer à prendre cette habitude maintenant!

#### Génération des ensembles d'entraînement ($80~\%$) et de test ($20~\%$)

Les données d'entraînement vont servir à entraînement le modèle de régression. C'est-à-dire, à estimer la valeur du seuil $\tau$ de chaque noeud, dans chaque arbre décisionnel de la forêt. Les données de test vont ensuite servir à mesurer les performances du modèle entrainé à prédire correctement le prix des maisons.

<!-- #endregion -->



In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)



<!-- #region id="e2fe1a4c" -->
#### Normalisation des données

Les paramètres de la fonction de normalisation sont calculés à partir des données d'**entraînement** uniquement! Elle est ensuite appliquée aux données d'entraînement et de test.
<!-- #endregion -->



In [ ]:
scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)
X_test_s = scaler.transform(X_test)



<!-- #region id="eba93605" -->
## Entraînement d'un régresseur par forêt aléatoire
#### Sélection du type de régresseur. 
Les valeurs des paramètres sont différentes de celles par défaut. Elles ont été optimisées afin 
de maximiser les performances du modèle. La méthode d'optimisation 
utilisée est expliquée dans le module sur la validation croisée.
<!-- #endregion -->



In [ ]:
reg = RandomForestRegressor(max_depth=6, min_samples_split=3, n_estimators=100)



<!-- #region id="eba93605" -->
#### Entraînement du régresseur avec les données d'entraînement
<!-- #endregion -->



In [ ]:
reg.fit(X_train_s, y_train);



<!-- #region id="cd54d0af" -->
#### Prédiction du prix des maisons pour les deux ensembles de données
<!-- #endregion -->



In [ ]:
y_train_pred = reg.predict(X_train_s)
y_test_pred = reg.predict(X_test_s)



<!-- #region id="e9d9263a" -->
## Affichage des résultats

#### Affichage de la valeur du R2

<!-- #endregion -->



In [ ]:
score = r2_score(y_train, y_train_pred)
print('Valeur de R2 en entraînement:', round(100 * score, 1), '%')

score = r2_score(y_test, y_test_pred)
print('Valeur de R2 en test:', round(100 * score, 1), '%')



<!-- #region id="97c90eef" -->
La valeur du $R^2$ correspond à la fraction de la variance des données qui est expliquée par le modèle utilisé.

Le modèle explique environ $95~\%$ de la variabilité des données en entraînement et $87~\%$ de celles en test.

#### Affichage des valeurs prédites et réelles des maisons pour l'ensemble de test

<!-- #endregion -->



In [ ]:
fig = plt.figure(figsize=(8, 6))
plt.plot(y_test, y_test_pred, 'o')
plt.plot(y_test, y_test, '-r', label='Prédiction parfaite')

plt.xlabel('Prix réels (k$)', fontsize=16)
plt.ylabel('Prix prédits (k$)', fontsize=16)
plt.legend()
fig.tight_layout()
plt.show()



<!-- #region id="8c1592f9" -->
La droite rouge correspond à une prédiction parfaite. Mises à part quelques données
aberrantes (*outliers*), il y a une bonne adéquation entre les résultats réels et prédits.

## Identification des variables les plus importantes déterminant le prix d'une maison

<!-- #endregion -->

<!-- #region id="c19b36a9" -->
<p>&nbsp;</p>
<div align="center">
    <img src= "../images/inspector-with-magnifying-glass.jpeg"  width="200" />
    <div>
    <font size="0.5">Image Source: http://clipart-library.com/clipart/1416328.htm</font>
    </div>
</div>
<!-- #endregion -->

<!-- #region id="8f7b5f5e" -->
Il est utile de faire de bonnes prédictions, mais il est aussi intéressant de savoir quelles sont les variables qui affectent le plus le prix des maisons. Les courtiers immobiliers aime prédire le prix d'une maison qu'ils ont à vendre. Les sociologues, eux, sont curieux de savoir comment les différentes variables socio-économiques affectent l'immobilier, et donc les conditions de vie, dans les différents quartiers d'une ville.

Les variables $x_{i}$ les plus importantes sont les plus utilisées pour prendre
des décisions à travers les arbres décisionnels constituant la forêt.

Le régresseur utilisé, RandomForestRegressor, permet de calculer l'importance de chaque variable. Celle-ci prend
une valeur entre 0 et 1, où 0 indique qu'elle n'est jamais utilisée et où 1 indique qu'elle est la seule utilisée
parmi toutes. Dans ce dernier cas, la variable permet de prédire parfaitement la réponse.
La somme des importances sur l'ensemble des variables vaut 1.

<!-- #endregion -->



In [ ]:
# Importance de chaque variable
importances = list(reg.feature_importances_)

# Associe le nom de chaque variable avec son importance
feature_importances = [
    (feature, round(importance, 2))
    for feature, importance in zip(feature_list, importances)
]

# Ordonne les valeurs d'importance en ordre décroissant
feature_importances = sorted(feature_importances, key=lambda x: x[1], reverse=True)



<!-- #region id="1d44429a" -->
### Affichage du diagrame d'importance des variables du jeu de données

On veut déterminer lesquelles sont les plus importantes et combien il y en a.
<!-- #endregion -->



In [ ]:
indices = np.argsort(importances)[::-1]

plt.style.use('fivethirtyeight')

var = list(range(len(importances)))
plt.bar(var, np.array(importances)[indices.astype(int)], orientation='vertical')
plt.xticks(var, np.array(feature_list)[indices.astype(int)], rotation='vertical')
plt.ylabel('Importance', fontsize=16)
plt.xlabel('Variable', fontsize=16)
plt.title('Importance des variables', fontsize=16)



<!-- #region id="f289b88c" -->
On voit que les variables RM (le nombre moyen de pièces par logement) et LSTAT (le pourcentage des classes défavorisées dans la population) sont de loin les plus importantes. Elles contribuent à $83~\%$ de l'importance totale; elles sont utilisées dans $83~\%$ des décisions des arbres de la forêt.

### Comment varie le prix des maisons en fonction des deux variables les plus importantes?

C'est bien de savoir quelles variables sont les plus déterminantes, c'est encore mieux de savoir comment chacune influence le prix des maisons. C'est ce que nous allons voir avec les deux variables principales.

#### Sélection des variables d'intérêt
<!-- #endregion -->



In [ ]:
var_int = ['RM', 'LSTAT']
x = X.loc[:, var_int]
x = pd.DataFrame(data=x, columns=var_int)



<!-- #region id="d6a5ccea" -->
#### Affichage de l'effet de chaque variable sur le prix des maisons

Dans chaque panneau, la droite des moindres carrés est affichée ainsi que son intervalle de confiance à $95~\%$.
<!-- #endregion -->



In [ ]:
fig, axs = plt.subplots(ncols=2, nrows=1, figsize=(10, 5))
index = 0
axs = axs.flatten()
for i, k in enumerate(var_int):
    u = pd.Series(x[k], name="x_var")
    v = pd.Series(y, name="Prix des maisons (k$)")
    sns.regplot(y=v, x=x[k], ax=axs[i])
plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=5.0)



<!-- #region id="6a003dc0" -->
On voit que le prix d'une maison augmente avec le nombre de chambres (RM) et diminue avec le
le pourcentage des classes défavorisées dans la population (LSAT). Une conclusion similaire
pourrait être tirée de bases de données contemporaines pour un très grand nombre de villes dans le monde.

<!-- #endregion -->

<!-- #region id="65bc2627" -->
## Exercice

Quelles seraient les performances en régression (la valeur du $R^2$) si la forêt aléatoire n'utilisait que
les deux variables les plus importantes RM et LSAT? 

Refaites l'exercice précédent avec les modifications suivantes:


- variables: RM et LSAT, 
- reg = RandomForestRegressor(max_depth=4, min_samples_split=5, n_estimators=100) 



Comment interprétez-vous les résultats?

**Réponse: Les deux valeurs de $R^2$ ont chuté. Il y a moins d'information disponible dans 2 variables que dans 13. 
Selon le diagramme de l'importance des variables, les deux variables les plus importantes permettent de prendre 
$83~\%$ des décisions dans une forêt. Les $17~\%$ restantes sont prises par les autres variables. En éliminant celles-ci de l'analyse, les performances en test ont chuté de $14~\%$. Ça montre à quel point les variables éliminées contribuaient initialement. Bien que les variables RM et LSAT soient les plus importantes, elles n'expliquent pas tout.**

<!-- #endregion -->



In [ ]:
# SOLUTION

# Génération des ensembles d'entraînement
X_train, X_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=42
)

# Normalisation des données
scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)
X_test_s = scaler.transform(X_test)

# Sélection du type de régresseur
reg = RandomForestRegressor(max_depth=4, min_samples_split=5, n_estimators=100)

# Entraînement du régresseur avec les données d'entraînement
reg.fit(X_train_s, y_train)

# Prédiction du prix des maisons pour les deux ensembles de données
y_train_pred = reg.predict(X_train_s)
y_test_pred = reg.predict(X_test_s)

# Affichage de la valeur du R2
score = r2_score(y_train, y_train_pred)
print('Valeur de R2 en entraînement:', round(100 * score, 1), '%')

score = r2_score(y_test, y_test_pred)
print('Valeur de R2 en test:', round(100 * score, 1), '%')



<!-- #region id="dd080c7f" -->
## Exercice

Affichez à nouveau les valeurs prédites et réelles du prix des maisons pour l'ensemble de test. En quoi la figure
est-elle différente de la version précédente?

**Réponse: les données sont maintenant plus dispersées autour de la droite unité. Si les 11 variables éliminées n'avaient aucun effet sur le prix des maisons, la dispersion des données aurait été comparable à celle dans la version précédente de la figure. Cela montre que les variables éliminées permettent d'affiner les prédictions du prix de vente des maisons.**

<!-- #endregion -->



In [ ]:
# SOLUTION

fig = plt.figure(figsize=(8, 6))
plt.plot(y_test, y_test_pred, 'o')
plt.plot(y_test, y_test, '-r', label='Prédiction parfaite')

plt.xlabel('Prix réels (k$)', fontsize=16)
plt.ylabel('Prix prédits (k$)', fontsize=16)
plt.legend()
fig.tight_layout()
plt.show()
